# Simulation of TLS handshake

In [1]:
""" # Please remove the triple quotes in this line if the pip installation of the libraries is needed
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
%pip install pycryptodome
%pip install cryptography
%pip install requests --disable-pip-version-check """;

**Client side**

The client generates a nonce of size 32 bytes. The client sends the nonce to the server along with a flag that is set to 1 if RSA will be used for key exchange, and set to zero if Diffie-Hellman will be used

In [2]:
import random
from Crypto.Random import get_random_bytes

# generate random nonce, size 32 bytes → 256-bit
nonce_client = get_random_bytes(32)    #RB
# specify exchange method → 0=Diffie-Hellman / 1=RSA
exchange_method_client=random.randint(0,1)
print("Nonce-client: ",nonce_client)
print("Exchange method-client:",exchange_method_client)

Nonce-client:  b'Wv\xeb{\x8er\x19\xa8\xb6\x13{U\xec+\x96\xe7\x8b\x9bL\x7f\xa0I[`\x94\xec\xaf\x8cm\x83\xb1<'
Exchange method-client: 0


**Server side**

The server responds with another nonce of size 32 bytes and a flag indicating the choice of key exchange algorithm.

In [3]:
# generate random nonce, size 32 bytes → 256-bit
nonce_server = get_random_bytes(32)     #RS
# specify exchange method → 0=Diffie-Hellman / 1=RSA
exchange_method_server=random.randint(0,1)
print("Nonce-server: ",nonce_server)
print("Exchange method-server:",exchange_method_server)

Nonce-server:  b'\xe9)\xa9\xef\xca\x95\xce\xec\x94\xf7\xb7\x83\xc1\xf0\x93v\x98\xa2\x15\xed\xdce\x91U\xe5\xa6\xee\x96.\x0c\xdc&'
Exchange method-server: 0


**Generate RSA KEY pair**

The server randomly generates a pair of RSA public/private keys and sends to the client {public key, hash(public key)}. This constitutes the certificate.

In [4]:
from Crypto.PublicKey import RSA
# generate RSA key pair
key = RSA.generate(2048)
publickey = key.publickey() # for later use
pu_key=key.publickey().exportKey()
priv_key=key.exportKey()
print(pu_key.decode())
# print(priv_key.decode())  #private key
# print(key.public_key().export_key().decode())
# print(f"Public key:  (n={hex(key.n)}, e={hex(key.e)})")
# print(f"Private key: (n={hex(key.n)}, d={hex(key.d)})")

-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA0eAW4lJaemagHPoDCwoK
0bwyH6w31Rw3Az0PGgdy4aOc7G/erg7rYXxJLIv4Lusz9RvuHDR4LmBRH7G8OKi7
qCTsdlvrhSejYFkRUDYYNQeAJ4C7A8kcBpomn3p99z0PbwMbIk6hB2fNlm3b3foN
DyJ7kkVAUSBZ7jas02J0wOe7Z0SyhZaz+ywVWBtKR6FGQhQ2+JN/7KxjUhUMu20M
RtreVNIQYQSuv93hN2DdZHxe7KG3PZs19X9RgyG+bEZSdlR/J+GAxeVDeiwzLMzr
+ii79/XFkmjIvE0lCZUsOYbIOYQ5Q01CirdD15cjzdpyu3pQmXCRin+I+wvefnKO
swIDAQAB
-----END PUBLIC KEY-----


**Key hashing-server**

In [5]:
import hashlib
from hashlib import sha256

# Hash key 
pk_client=key.public_key().export_key()
my_sha = hashlib.sha256(pk_client)
hash_client = int.from_bytes(my_sha.digest(),byteorder='big')
print("Sent to client: {public key, hash(public key)} ",'\n'+'↓↓↓↓↓↓↓↓↓↓↓↓','\n')
print("Public key :",'\n',pk_client.decode(),'\n')
print("Generated hash :",'\n',hash_client)
# signature = pow(hash, key.d, key.n)
# print("Signature:", hex(signature))

Sent to client: {public key, hash(public key)}  
↓↓↓↓↓↓↓↓↓↓↓↓ 

Public key : 
 -----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA0eAW4lJaemagHPoDCwoK
0bwyH6w31Rw3Az0PGgdy4aOc7G/erg7rYXxJLIv4Lusz9RvuHDR4LmBRH7G8OKi7
qCTsdlvrhSejYFkRUDYYNQeAJ4C7A8kcBpomn3p99z0PbwMbIk6hB2fNlm3b3foN
DyJ7kkVAUSBZ7jas02J0wOe7Z0SyhZaz+ywVWBtKR6FGQhQ2+JN/7KxjUhUMu20M
RtreVNIQYQSuv93hN2DdZHxe7KG3PZs19X9RgyG+bEZSdlR/J+GAxeVDeiwzLMzr
+ii79/XFkmjIvE0lCZUsOYbIOYQ5Q01CirdD15cjzdpyu3pQmXCRin+I+wvefnKO
swIDAQAB
-----END PUBLIC KEY----- 

Generated hash : 
 52142314751088137259342536457898036324029300136928977175865715230573016818391


In [6]:
# hash2 = int.from_bytes(my_sha.digest(),byteorder='big')
# hashFromSignature = pow(signature, key.e, key.n)
# print("Signature valid:", hash2 == hashFromSignature)

**Key re-hashing-client**

Client re-hashes the public key and and compares with the hash that is sent. If they match the client proceeds to Step 5. Otherwise, the process ends here with an error message.

In [7]:
# re-Hash key 
my_sha = hashlib.sha256(pk_client)
hash_server = int.from_bytes(my_sha.digest(),byteorder='big')
if (hash_client==hash_server):
    print("hash match")
else:
    print("hash missmatch")

hash match


# RSA-TLS

In case RSA was chosen, then the client will generate a pre-master secret and encrypt it with the public key of the server that was already sent over by the server.

In RSA key exchange the public key is used in the client to encrypt the pre-master secret and the private key is used in the server to decrypt it.

In [9]:
from Crypto.Cipher import PKCS1_OAEP
# encrypt the Pre-master secret with the public key of the server
encryptor = PKCS1_OAEP.new(publickey)
pre_masterSecret = get_random_bytes(32)     #PS
encrypted = encryptor.encrypt(pre_masterSecret)
print(encrypted)


b'1\xf1\xc5\xb3=\xa9]6\xbd\xa7\xd9\xdb\xab\x98\x00\x04\xe8\x00\xe4(_y\x17\x0f\xe5D\xa4\x87z\x1eR\xae\xb65\xfe\x12\xa4k\x84\xf4O\x13\x89\x90l\xbbj\x88\xe5\xd8j\xa7\x89\\\xcb\xce\x7f\xd0\xa2?f2h\xa2=.\x7f)\xdfW\xb8\x06m\x84\xde(\x01\x90\x89P\x1e:\xae\xa3\x03O\xf5\xa5\xfb\x15\xbf\xa4\x05\xf9\x85\x1c\xa5x>\xfc\x0b\xe7\xd0\n\xe1\xaf&id\xe7\xb4\xb2\x87\x1fo\x86\xc5\xf8\xc4\xfaK\xe0\x95\x9f\x1e\xf4t\x8f\xfdE\xc5/\xe3V]\x98pp\x03\xfa\xea\x90\xc8\xf1\xb6L\x8a\xdd\xdb\xe7\xf2\x1bo\xbd\xf6\xa5\xdb\xfa\xcb\x853\tYf\xde\xb5"\x963r7\x07\xdc\xe1W\xc9{sb\x95\xd3\xc6\xcc~\xfe\xbc\x82\xc9\x85\xa7`r\xc6\x03\xfa\x1bA\xa8t.\x93\xbc\xb9gq;\xbc\x13k \n\x8f\xad\xf1*9\xaa\xb7\xa6\xfc\xd0\x14\xf8F\xa8K\x08\xca\xbd\xf3\x80\x9f\xb81\xdbK\xdf\xf0\xbc\x9c\xf4\x93\xbc\xb2\xc5\x0c\x18\x063\xe3\xda\x10\x9f+w\xfb'


# Diffie-Hellman TLS